# 2 - Training a Model on Remote Compute

In this notebook, you will train a model on remote compute powered by Azure Machine Learning. While we will be working primarily in this Jupyter notebook, the actual training will be done on a separate compute cluster.

## Imports

We first need to import several modules from the Azure Machine Learning Python SDK:

In [3]:
import os
import urllib
import shutil
import azureml
import azureml.core
from azureml.core import Workspace, Run
from azureml.core import Experiment
from azureml.core.compute import AmlCompute
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.dnn import TensorFlow
from azureml.widgets import RunDetails
from azureml.opendatasets import MNIST
from azureml.core import dataset
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.33.0


## Workspace

If you are running this notebook as a part of your Azure Machine Learning Studio workspace, you can get a reference to your workspace by simply running the following code:

In [5]:
ws = Workspace.from_config()
print("Azure ML Workspace")
print(f'Name: {ws.name}')
print(f'Location: {ws.location}')
print(f'Resource Group: {ws.resource_group}')

Azure ML Workspace
Name: carlartest
Location: westeurope
Resource Group: networkwatcherrg


## Provisioning Compute Resources

We will include the logic from the previous module around provisioning compute resources.  This will create a cluster of Standard NC6 instances that can scale from 0 to 4 nodes.

In [6]:
# Create a name for our new cluster
cpu_cluster_name = 'Std-NC24-Clus'

# Verify that cluster does not exist already
try:
    cpu_cluster = AmlCompute(workspace=ws, name=cpu_cluster_name)
    print('Cluster already exists.')
    
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC24',
                                                           max_nodes=4)
    cpu_cluster = AmlCompute.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Create an Experiment

In [7]:
# Get reference to the workspace
ws = Workspace.from_config()
print("Azure ML Workspace")
print(f'Name: {ws.name}')
print(f'Location: {ws.location}')
print(f'Resource Group: {ws.resource_group}')

# Create an experiment, or get a reference to the experiment if it already exists
experiment_name = 'cutie'
exp = Experiment(workspace=ws, name=experiment_name)
print("Azure ML Experiment")
print(f'ID: {exp.id}')
print(f'Name: {exp.name}')

Azure ML Workspace
Name: carlartest
Location: westeurope
Resource Group: networkwatcherrg
Azure ML Experiment
ID: cf203022-9828-45f5-971e-34425bfa0826
Name: cutie


Next, we will create a directory that will hold the script we will use for training our model.  This directory will eventually be sent to our remote compute cluster:

In [8]:
import os
script_folder = os.path.join(os.getcwd(), "cutie")
os.makedirs(script_folder, exist_ok=True)

Next, we need to output the script that we will use to train our model. 

Within an experiment in Azure Machine Learning, each time you execute a test on your experiment, it is considered a `Run`.  The documentation defines this as:

> A run represents a single trial of an experiment. Runs are used to monitor the asynchronous execution of a trial, log metrics and store output of the trial, and to analyze results and access artifacts generated by the trial. [Run Class](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.run(class)?view=azure-ml-py)

We will need to get the context of the current `Run` from within our training script. We can use this to register metrics from our test and make them associated with that specific `Run`.

Since our training script will also need to access our model source code, we will also copy it into this directory:

In [9]:
import distutils.dir_util
distutils.dir_util.copy_tree('Codebase', script_folder)

['/mnt/batch/tasks/shared/LS_root/mounts/clusters/carlartest/code/Users/fiabebe/CutieTest/cutie/.amlignore',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/carlartest/code/Users/fiabebe/CutieTest/cutie/.amlignore.amltmp',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/carlartest/code/Users/fiabebe/CutieTest/cutie/.gitignore',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/carlartest/code/Users/fiabebe/CutieTest/cutie/.idea/CUTIE.iml',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/carlartest/code/Users/fiabebe/CutieTest/cutie/.idea/misc.xml',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/carlartest/code/Users/fiabebe/CutieTest/cutie/.idea/modules.xml',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/carlartest/code/Users/fiabebe/CutieTest/cutie/.idea/vcs.xml',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/carlartest/code/Users/fiabebe/CutieTest/cutie/.idea/workspace.xml',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/carlartest/code/Users/fiabebe/CutieTest/cut

## Create a custom environment

In [10]:
tf_env = Environment.from_conda_specification(name='tensorflow-2.2-gpu', file_path='./conda_dependencies.yml')

# Specify a GPU base image
tf_env.docker.enabled = True
tf_env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04'

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


## Create a ScriptRunConfig for Training

Create a ScriptRunConfig object to specify the configuration details of your training job, including your training script, environment to use, and the compute target to run on. Any arguments to your training script will be passed via command line if specified in the arguments parameter.

In [11]:
from azureml.core import ScriptRunConfig

args = ['--load_dict', True,
       '--doc_path','invoice_data']

src = ScriptRunConfig(source_directory=script_folder,
                      script='main_train_json.py',
                      arguments=args,
                      compute_target=cpu_cluster,
                      environment=tf_env)

## Submit a training run
The Run object provides the interface to the run history while the job is running and after it has completed.

In [12]:
run = Experiment(workspace=ws, name='CUTIE-Run').submit(src)
run.wait_for_completion(show_output=True)

RunId: CUTIE-Run_1632143963_755a282a
Web View: https://ml.azure.com/runs/CUTIE-Run_1632143963_755a282a?wsid=/subscriptions/3b861ce0-dd31-4dff-8de6-79240859c730/resourcegroups/networkwatcherrg/workspaces/carlartest&tid=72f988bf-86f1-41af-91ab-2d7cd011db47

Streaming azureml-logs/55_azureml-execution-tvmps_7657f941f5030d5de19a2b54dbcbefdd193488c1a0f94de19fa054b19cf19f42_d.txt

2021-09-20T13:27:10Z Running following command: /bin/bash -c sudo blobfuse /mnt/batch/tasks/shared/LS_root/jobs/carlartest/azureml/cutie-run_1632143963_755a282a/mounts/workspaceblobstore --tmp-path=/mnt/batch/tasks/shared/LS_root/jobs/carlartest/azureml/cutie-run_1632143963_755a282a/caches/workspaceblobstore -o ro --file-cache-timeout-in-seconds=1000000 --cache-size-mb=1366173 -o nonempty -o allow_other --config-file=/mnt/batch/tasks/shared/LS_root/jobs/carlartest/azureml/cutie-run_1632143963_755a282a/configs/workspaceblobstore.cfg --log-level=LOG_WARNING
2021-09-20T13:27:10Z Successfully mounted a/an Blobfuse File

Since this will take some time to complete, we can use an included utility to see the progress within our Jupyter notebook:

In [15]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

Once it completes, we can extract the metrics from this run:

In [14]:
print(run.get_metrics())

{}


## Registering Model

Finally, we can register our model in our workspace, so that it can be leveraged elsewhere.  We will use this model in the next clip to deploy it as a web service.

In [17]:
# register model 
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model = run.register_model(model_name='CUTIE', 
                           model_path='Codebase/meals/CUTIE_highresolution_8x_d20000c9(r80c80)_iter_40000.ckpt',
                           model_framework=Model.Framework.TENSORFLOW,
                           model_framework_version='1.12.0',
                           resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5))

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path Codebase/meals/CUTIE_highresolution_8x_d20000c9(r80c80)_iter_40000.ckpt in the set of files uploaded to the run: ['azureml-logs/55_azureml-execution-tvmps_450b75472978d275752a69a1cde86dc1753c02317bc2d5e38c8d66dcb2672a08_d.txt', 'azureml-logs/65_job_prep-tvmps_450b75472978d275752a69a1cde86dc1753c02317bc2d5e38c8d66dcb2672a08_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json']
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Could not locate the provided model_path Codebase/meals/CUTIE_highresolution_8x_d20000c9(r80c80)_iter_40000.ckpt in the set of files uploaded to the run: ['azureml-logs/55_azureml-execution-tvmps_450b75472978d275752a69a1cde86dc1753c02317bc2d5e38c8d66dcb2672a08_d.txt', 'azureml-logs/65_job_prep-tvmps_450b75472978d275